In [1]:
import numpy as np
import pandas as pd

import ast
import random
import lmstudio as lms

from IPython.display import Markdown

In [2]:
raw_link = pd.read_csv('../Data/postgenerate_gpt.csv')
raw_tthc = pd.read_csv('../Data/raw_tthc.csv', dtype={'maThuTuc': str})

In [3]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [4]:
raw_tthc.columns = cols

In [5]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [6]:
example = []

example.append("""
#Câu hỏi#: Giống chó hiếm nào được phát triển như một biến thể của Rat Terrier, Shiloh Shepherd hoặc American Hairless Terrier?
#Câu trả lời#: American Hairless Terrier
#Đánh giá của bạn#: Không

""")

example.append("""
#Câu hỏi#: Các kênh thoát nước của New Orleans có cùng độ dài với kênh Augusta không?
#Câu trả lời#: Không, các kênh thoát nước của New Orleans và kênh Augusta không có cùng độ dài. Kênh Orleans dài khoảng 3,6 dặm (5,8 km) trong khi kênh Augusta dài khoảng 7 dặm (11,3 km).
#Đánh giá của bạn#: Có

#Câu hỏi#: Tuyến đường cao tốc Hoa Kỳ nào cho phép tiếp cận Zilpo Road và cũng được gọi là Midland Trail?
#Câu trả lời#: U.S Highway 70
#Đánh giá của bạn#: Có

""")

example.append("""
#Câu hỏi#: Superheaven và Oceansize thuộc thể loại nhạc nào?
#Câu trả lời#: Superheaven và Oceansize thuộc thể loại nhạc rock.
#Đánh giá của bạn#: Không

#Câu hỏi#: Kōbō Abe và Agatha Christie có chung nghề nghiệp nào?
#Câu trả lời#: Nhà viết kịch.
#Đánh giá của bạn#: Không

""")

example.append("""
#Câu hỏi#: Ban nhạc nào có nhiều thành viên hơn, Muse hay The Raconteurs?
#Câu trả lời#: Muse có nhiều thành viên hơn The Raconteurs.
#Đánh giá của bạn#: Có

#Câu hỏi#: Mỏ nào hiện có giá trị hơn, Temagami-Lorrain Mine hay Meadowbank Gold Mine?
#Câu trả lời#: Meadowbank Gold Mine, vì mỏ Meadowbank vẫn đang khai thác vàng trong khi mỏ Temagami-Lorrain đã ngừng hoạt động nhiều năm.
#Đánh giá của bạn#: Không

""")

In [7]:
config = {"temperature": 0.7, "maxTokens": 32, "topPSampling": 0.9}

In [8]:
# evaluate_template = \
# """Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi và câu trả lời, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không. Bạn NÊN đưa ra đánh giá dựa trên các loại ảo giác (hallucination) sau đây và kiến thức thực tế.
# {pattern}

# #Câu hỏi#: {question}

# #Câu trả lời#: {answer}

# #Đánh giá của bạn#:

# Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. Câu trả lời bạn đưa ra PHẢI là "Có" hoặc "Không".
# """

In [9]:
evaluate_context = """Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi và câu trả lời, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không.
Câu trả lời bạn đưa ra bắt buộc CHỈ là "Có" hoặc "Không", và không giải thích gì thêm. Trả lời "Có" nếu câu trả lời chứa thông tin ảo giác, trả lời "Không" nếu câu trả lời không chứa thông tin ảo giác.
{pattern}
Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. 
"""

In [10]:
evaluate_template = \
"""
#Câu hỏi#: {question}

#Câu trả lời#: {answer}

#Đánh giá của bạn#:
"""

In [ ]:
model_name = "falcon-mamba-7b-instruct"

filename = f'{model_name}_evaluate.csv'

model = lms.llm(model_name)

cols = ['link', 'cauTraLoiDung', 'danhGiaDung', 'cauTraLoiAoGiac', 'pattern', 'danhGiaAoGiac']

try:
    df = pd.read_csv(filename)
    if df.empty:
        print("CSV file is empty")
        df = pd.DataFrame(columns=cols)
except FileNotFoundError:
    print(f"File not found: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)
except pd.errors.EmptyDataError:
    print(f"File is completely empty: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)

for i in range(len(raw_link)):
    if raw_link['link'][i] in list(df['link'].values):
        continue

    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    right_answer = raw_link['cauTraLoi'][i]
    hallucinated_answer = raw_link['cauTraLoiAoGiac'][i]

    context_right = evaluate_context.format(pattern="")
    context_hallucinated = evaluate_context.format(pattern=pattern[p])

    prompt_right = evaluate_template.format(question=question, answer=hallucinated_answer)
    prompt_hallucinated = evaluate_template.format(question=question, answer=hallucinated_answer)

    try:
        completion = model.respond(history={"messages": [
            {"role": "system", "content": context_right},
            {"role": "user", "content": prompt_right}
        ]}, config=config)
        output_right = str(completion)
        completion = model.respond(history={"messages": [
            {"role": "system", "content": context_hallucinated},
            {"role": "user", "content": prompt_hallucinated}
        ]}, config=config)
        output_hallucinated = str(completion)
    except Exception as e:
        output_right = ""
        output_hallucinated = ""
        print(e)
        break

    results = [raw_link['link'][i], right_answer, output_right, hallucinated_answer, p, output_hallucinated]
    df.loc[i, cols] = results    
    df.to_csv(filename, index=False)

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

File is completely empty: falcon-mamba-7b-instruct_evaluate.csv


{"channel_id": 11, "event": "Received unhandled message {'type': 'channelSend', 'channelId': 11, 'message': {'type': 'fragment', 'fragment': {'content': '\u0301', 'tokensCount': 1, 'containsDrafted': False, 'reasoningType': 'none'}}} for already closed channel", "ws_url": "ws://localhost:1234/llm"}
{"channel_id": 11, "event": "Received unhandled message {'type': 'channelSend', 'channelId': 11, 'message': {'type': 'fragment', 'fragment': {'content': ' d', 'tokensCount': 1, 'containsDrafted': False, 'reasoningType': 'none'}}} for already closed channel", "ws_url": "ws://localhost:1234/llm"}
{"channel_id": 11, "event": "Received unhandled message {'type': 'channelSend', 'channelId': 11, 'message': {'type': 'fragment', 'fragment': {'content': 'ung', 'tokensCount': 1, 'containsDrafted': False, 'reasoningType': 'none'}}} for already closed channel", "ws_url": "ws://localhost:1234/llm"}


KeyboardInterrupt: 

{"channel_id": 11, "event": "Received unhandled message {'type': 'channelSend', 'channelId': 11, 'message': {'type': 'fragment', 'fragment': {'content': ' lu', 'tokensCount': 1, 'containsDrafted': False, 'reasoningType': 'none'}}} for already closed channel", "ws_url": "ws://localhost:1234/llm"}
{"channel_id": 11, "event": "Received unhandled message {'type': 'channelSend', 'channelId': 11, 'message': {'type': 'fragment', 'fragment': {'content': '\u00f4', 'tokensCount': 1, 'containsDrafted': False, 'reasoningType': 'none'}}} for already closed channel", "ws_url": "ws://localhost:1234/llm"}
{"channel_id": 11, "event": "Received unhandled message {'type': 'channelSend', 'channelId': 11, 'message': {'type': 'fragment', 'fragment': {'content': 'ng', 'tokensCount': 1, 'containsDrafted': False, 'reasoningType': 'none'}}} for already closed channel", "ws_url": "ws://localhost:1234/llm"}
{"channel_id": 11, "event": "Received unhandled message {'type': 'channelSend', 'channelId': 11, 'message':

In [ ]:
# Dead kernel by interrupting so re-importing

import pandas as pd
from IPython.display import Markdown
import random

model_name = "falcon-mamba-7b-instruct"

filename = f'{model_name}_evaluate.csv'

df = pd.read_csv(filename)

display(Markdown(df['danhGiaDung'][random.randint(0, len(df)-1)]))
print()
display(Markdown(df['danhGiaAoGiac'][random.randint(0, len(df)-1)]))

Có

Có